In [5]:
import csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

csv.field_size_limit(100000000)

ModuleNotFoundError: No module named 'sklearn'

In [4]:
def parse_csv(filename):
    data = []
    with open(filename, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        next(reader)  # Skip the header row
        for row in reader:
            email_text = row[1].replace('""', '"')  # Replace double double quotes with a single quote
            email_type = row[2]
            data.append({'Email Text': email_text, 'Email Type': email_type})
    return data

# Example usage
filename = 'data/Phishing_Email.csv'
parsed_data = parse_csv(filename)

print(len(parsed_data))


18650
